In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn
from chinese_calendar import is_workday

In [2]:
# 先製造日期在計算工作日，接著在和原本資料合併

In [3]:
def work_day(data,year):
    check_date = pd.read_csv(f"./週期資料.csv",encoding='utf-8')
    answer =  pd.DataFrame()
    date =  pd.DataFrame()
    date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))
    date['週'] = date['日期'].dt.isocalendar().week
    date['年'] = date['日期'].dt.isocalendar().year
    date = date.drop(date[date['週'] == 53].index).reset_index()
    del_data =  date[date['週'] == 52].tail(1).index
    date = date.drop(date[del_data.values[0]+1:].index).reset_index()
    date["日期"] = pd.to_datetime(date["日期"] ,format='%Y/%m/%d')
    check_date["date"] = pd.to_datetime(check_date["date"] ,format='%Y/%m/%d')
    # 如果是工作日就存成true，否則存成false
    date['休假日']=date['日期'].map(lambda x:(check_date['date']==x).any())
    print(date)

    for num in range(1, 53,1):
        temp = date[date['週'] == num]
#         計算該周工作日
        holiday = len(temp[temp['休假日'] == True])
        work_day = len(temp[temp['休假日'] == False])
        date.loc[date['週']==num,'工作天數'] = work_day
        date.loc[date['週']==num,'休假天數'] = holiday
        if year == 2021 and num == 52:   
            date.loc[date['週']==num,'休假天數'] = 2

#     同一週期工作日、休假日都一樣所以只保留一筆
    date.drop_duplicates(subset='週', keep='last', inplace=True)
    date = date.drop(['日期', '休假日','level_0','index'], axis=1)
  
#     answer = pd.merge(data, date, on='週',how='outer')

    answer = pd.merge(data, date, on='週')
    answer = answer.sort_values(['帳務日期'], ascending=True).reset_index(drop=True)

#     刪除日期和工作日

    
    return answer

讀取資料+整理工作日資料

In [5]:
# B03106859
# C13930399
# A00120213
# A02120180
# A02320340
# A02322259
# A04411285
# A04800036
# B00206057 50週
# B03110120
# G81500272
# K80004044

part_no = 'K82430329'
# 病房
room = '五病房'
# 年分
years = [2019,2020,2021,2022]

total_data =  pd.DataFrame()

for year in years:
    data = pd.read_csv(f"../使用量計算/全部資料合併/{year}_{room}_合併檔.csv",encoding='utf_8_sig')
    data = data[data.columns.drop(list(data.filter(regex='Unnamed')))].reset_index(drop=True)
    data = data[data['料號'] == part_no]
    data = data.drop(data[data['數量'] < 0].index).reset_index()
    data = data.sort_values(['週'], ascending=True).reset_index(drop=True)
    # 先刪除大於52周的資料
    del_data =  data[data['週'] == 52].tail(1).index
#     有可能沒有52週的資料
    if del_data.empty == False:
        data = data.drop(data[del_data.values[0]+1:].index).reset_index()
   
    # 計算工作日
    data = work_day(data,year)
    print("檢查",data)
    total_data = pd.concat([total_data, data])
    #     刪除多的欄位
total_data = total_data[total_data.columns.drop(list(total_data.filter(regex='index')))].reset_index(drop=True)
total_data = total_data[total_data.columns.drop(list(total_data.filter(regex='level_0')))].reset_index(drop=True)
# 把相同的值補上
temp = data.head(1)
total_data['庫房'] = temp['庫房'].values[0]
total_data['料號'] = temp['料號'].values[0]
total_data['品名'] = temp['品名'].values[0]
total_data['數量'].fillna(0,inplace=True)
total_data = total_data.sort_values(['年','週'], ascending=True).reset_index(drop=True)
# total_data.to_csv(f"./有加工作日/{room}/{part_no}.csv",encoding='utf_8_sig')
# total_data.to_csv(f"../一個月預測一次/資料集/{room}/{part_no}.csv",encoding='utf_8_sig')
# total_data.to_csv(f"../差值/2週資料集/{room}/{part_no}.csv",encoding='utf_8_sig')
# total_data.to_csv(f"../差值/1個月資料集/{room}/{part_no}.csv",encoding='utf_8_sig')


C:\Users\IDSL\AppData\Local\Temp\ipykernel_29612\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


     level_0  index         日期   週     年    休假日
0          0      0 2019-01-01   1  2019   True
1          1      1 2019-01-02   1  2019  False
2          2      2 2019-01-03   1  2019  False
3          3      3 2019-01-04   1  2019  False
4          4      4 2019-01-05   1  2019   True
..       ...    ...        ...  ..   ...    ...
358      358    358 2019-12-25  52  2019  False
359      359    359 2019-12-26  52  2019  False
360      360    360 2019-12-27  52  2019  False
361      361    361 2019-12-28  52  2019   True
362      362    362 2019-12-29  52  2019   True

[363 rows x 6 columns]
檢查    index    庫房         料號     品名   數量        帳務日期   週   工作日     年  工作天數  休假天數
0  15173  DLN5  K82430329  痰盒-綠蓋  1.0  2019-04-04  14  True  2019   3.0   4.0
1  15174  DLN5  K82430329  痰盒-綠蓋  1.0  2019-04-04  14  True  2019   3.0   4.0
2  25742  DLN5  K82430329  痰盒-綠蓋  1.0  2019-07-02  27  True  2019   5.0   2.0
3  29450  DLN5  K82430329  痰盒-綠蓋  1.0  2019-08-21  34  True  2019   5.0   2.0
4  2945

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29612\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


檢查     level_0  index    庫房         料號     品名   數量        帳務日期   週    工作日     年  \
0         0   5599  DLN5  K82430329  痰盒-綠蓋  1.0  2020-02-20   8   True  2020   
1         1  10581  DLN5  K82430329  痰盒-綠蓋  1.0  2020-03-03  10   True  2020   
2         2  10582  DLN5  K82430329  痰盒-綠蓋  1.0  2020-03-03  10   True  2020   
3         3  10583  DLN5  K82430329  痰盒-綠蓋  1.0  2020-03-03  10   True  2020   
4         4  10584  DLN5  K82430329  痰盒-綠蓋  1.0  2020-03-10  11   True  2020   
5         5  10585  DLN5  K82430329  痰盒-綠蓋  1.0  2020-03-24  13   True  2020   
6         6  21185  DLN5  K82430329  痰盒-綠蓋  2.0  2020-05-01  18  False  2020   
7         7  21186  DLN5  K82430329  痰盒-綠蓋  1.0  2020-05-07  19   True  2020   
8         8  21187  DLN5  K82430329  痰盒-綠蓋  1.0  2020-05-07  19   True  2020   
9         9  25785  DLN5  K82430329  痰盒-綠蓋  1.0  2020-06-09  24   True  2020   
10       10  30453  DLN5  K82430329  痰盒-綠蓋  1.0  2020-07-23  30   True  2020   
11       11  30454  DLN5  K82430329  

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29612\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


     level_0  index         日期   週     年    休假日
0          0      0 2022-01-01  52  2021   True
1          1      1 2022-01-02  52  2021   True
2          2      2 2022-01-03   1  2022  False
3          3      3 2022-01-04   1  2022  False
4          4      4 2022-01-05   1  2022  False
..       ...    ...        ...  ..   ...    ...
360      360    360 2022-12-27  52  2022  False
361      361    361 2022-12-28  52  2022  False
362      362    362 2022-12-29  52  2022  False
363      363    363 2022-12-30  52  2022  False
364      364    364 2022-12-31  52  2022   True

[365 rows x 6 columns]
檢查     index    庫房         料號     品名  數量        帳務日期   週    工作日     年  工作天數  休假天數
0   54681  DLN5  K82430329  痰盒-綠蓋   1   2022/1/11   2   True  2022   5.0   2.0
1   54703  DLN5  K82430329  痰盒-綠蓋   1  2022/10/13  41   True  2022   4.0   3.0
2   54704  DLN5  K82430329  痰盒-綠蓋   1  2022/10/27  43   True  2022   5.0   2.0
3   54706  DLN5  K82430329  痰盒-綠蓋   1   2022/11/1  44   True  2022   5.0   2.0
4 

C:\Users\IDSL\AppData\Local\Temp\ipykernel_29612\3802451557.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  date['日期'] = pd.DataFrame(pd.date_range(f'1/1/{year}',f'31/12/{year}'))


In [ ]:
#抓取日歷國定假日
import requests

page = 0
isHoliday = pd.DataFrame()
while True:
    url = f"https://data.ntpc.gov.tw/api/datasets/308DCD75-6434-45BC-A95F-584DA4FED251/json?page={page}&size=1000"
    res = requests.get(url)
    resJson = res.json()
    resJson2 = pd.DataFrame(resJson)
    isHoliday = pd.concat([isHoliday, resJson2])
    page+=1
    if resJson[-1]['date']>'2023':
        break

In [ ]:
isHoliday

In [ ]:
isHoliday
isHoliday['date'] = pd.to_datetime(isHoliday['date'])
isHoliday['isholiday']=isHoliday.apply(lambda row: 1 if row['isholiday']=='是' else 0, axis=1)
#1為國定假日0則不是
isHoliday = isHoliday[['date','isholiday']]
# isHoliday['Date'] = pd.to_datetime(isHoliday['Date'])
isHoliday = isHoliday[(isHoliday['date']<'2023')&(isHoliday['date']>='2019')]

isHoliday['週'] = isHoliday['date'].dt.isocalendar().week
# isHoliday['年'] = isHoliday['date'].dt.isocalendar().year
# isHoliday['月'] = isHoliday['date'].dt.month
# isHoliday[(isHoliday['年'] == 2021)&(isHoliday['月'] == 2)]
# isHoliday.to_csv(f"./週期資料.csv")
isHoliday

In [ ]:
total_data.tail(1500)